In [2]:
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/mariapavlou/Downloads/comp-548-ad7a1db0f762.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/mariapavlou/Downloads/comp-548-ad7a1db0f762.json


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade google-cloud-firestore

In [9]:
from google.cloud import firestore

# Initialize the Firestore client using the service account key file
db = firestore.Client.from_service_account_json("Users/mariapavlou/Downloads/comp-548-ad7a1db0f762.json")


In [10]:
def add_user(user_id, name, email):
    user_ref = db.collection("users").document(user_id)
    user_ref.set({
        "name": name,
        "email": email
    })

add_user("user_1", "Danae", "danae@example.com")
add_user("user_2", "Nick", "nick@example.com")



In [11]:
def add_document(document_id, title, created_at):
    doc_ref = db.collection("documents").document(document_id)
    doc_ref.set({
        "title": title,
        "created_at": created_at
    })

def add_collaborator(document_id, user_id, added_at):
    collab_ref = db.collection("documents").document(document_id).collection("collaborators").document(user_id)
    collab_ref.set({
        "user_id": user_id,
        "added_at": added_at
    })

def add_edit(document_id, user_id, edit_timestamp, content_added):
    edit_ref = db.collection("documents").document(document_id).collection("edits").document()
    edit_ref.set({
        "user_id": user_id,
        "edit_timestamp": edit_timestamp,
        "content_added": content_added
    })

add_document("doc_1", "Project Proposal", "2024-11-01T12:00:00Z")

add_collaborator("doc_1", "user_1", "2024-11-02T15:00:00Z")
add_collaborator("doc_1", "user_2", "2024-11-03T16:00:00Z")


add_edit("doc_1", "user_1", "2024-11-02T10:00:00Z", "Initial content of the document.")
add_edit("doc_1", "user_2", "2024-11-03T11:00:00Z", "Additional content added by another user.")


In [12]:
def get_collaborators(document_id):
    collab_ref = db.collection("documents").document(document_id).collection("collaborators")
    docs = collab_ref.stream()
    collaborators = []
    for doc in docs:
        collaborators.append(doc.to_dict())
    return collaborators


print("Collaborators for doc_1:", get_collaborators("doc_1"))


Collaborators for doc_1: [{'added_at': '2024-11-02T15:00:00Z', 'user_id': 'user_1'}, {'added_at': '2024-11-03T16:00:00Z', 'user_id': 'user_2'}]


In [13]:
def get_edits(document_id):
    edits_ref = db.collection("documents").document(document_id).collection("edits")
    docs = edits_ref.stream()
    edits = []
    for doc in docs:
        edits.append(doc.to_dict())
    return edits

print("Edits for doc_1:", get_edits("doc_1"))


Edits for doc_1: [{'user_id': 'user_1', 'content_added': 'Initial content of the document.', 'edit_timestamp': '2024-11-02T10:00:00Z'}, {'user_id': 'user_2', 'content_added': 'Additional content added by another user.', 'edit_timestamp': '2024-11-03T11:00:00Z'}]


In [14]:
def get_user_edits(document_id, user_id):
    edits_ref = db.collection("documents").document(document_id).collection("edits")
    query = edits_ref.where("user_id", "==", user_id)
    docs = query.stream()
    user_edits = []
    for doc in docs:
        user_edits.append(doc.to_dict())
    return user_edits

print("Edits by user_1 on doc_1:", get_user_edits("doc_1", "user_1"))


Edits by user_1 on doc_1: [{'user_id': 'user_1', 'content_added': 'Initial content of the document.', 'edit_timestamp': '2024-11-02T10:00:00Z'}]


/Users/mariapavlou/anaconda3/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
